In [36]:
%%writefile app.py

import streamlit as st

##Titulo global del app
st.image('./logo.png', width= 500)


#Menu del streamlit
menu = ['Inicio', 'Informes UA']

choice = st.sidebar.selectbox('Menu', menu)


if choice == 'Inicio':
    st.markdown('### Conjunto de scripts automatizados para TP informatica')

elif choice == 'Informes UA':
    
    import io
    import pandas as pd
    import numpy as np
    from openpyxl import load_workbook
    
    listado = st.file_uploader("Listado de facturas (xlsx)", type=["xlsx"])
    plantilla = st.file_uploader("Plantilla (xlsx)", type=["xlsx"])
    
    if (listado is not None) and (plantilla is not None):

        #Cargamos el listado del gestor de facturas
        listado = pd.read_excel(listado, skiprows=9)

        #Limpiar campos vacios
        listado.dropna(axis = 1, how='all', inplace = True)
        clean = listado.iloc[:, 1:].copy()
        clean['Unnamed: 7'] = clean['Unnamed: 7'].replace('.', np.nan)
        clean['Unnamed: 7'] = clean['Unnamed: 7'].replace('Artículo', np.nan)
        clean.dropna(subset = ['Unnamed: 7'], inplace = True)
        clean['Unnamed: 2'] = clean['Unnamed: 2'].astype(str).str[:-2]

        #Cambio column names
        cols = ['Nº CLIENTE', 'NOMBRE CLIENTE', 'REFERENCIA ARTÍCULO', 
            'DESCRIPCIÓN ARTÍCULO', 'CANTIDAD SERVIDA', 'PRECIO UNITARIO',
            'IMPORTE', 'Nº FACTURA', 'FECHA']

        clean.columns = cols
        clean.reset_index(drop=True, inplace = True)


        cols_num = ['CANTIDAD SERVIDA', 'PRECIO UNITARIO', 'IMPORTE']
        for col in cols_num:
            clean[col] = clean[col].str.replace('.', '')
            clean[col] = clean[col].str.replace(',', '.').astype(float)

        #Calculo nuevas columnas necesarias para el Report UA
        new_cols = ['UNIDAD DE VENTA', 'IMPORTE IVA', 'TOTAL VENTA', 'FECHA DE PEDIDO']
        for col in new_cols:
            clean[col] =pd.Series([], dtype=float)


        cols_ord = ['Nº CLIENTE', 'NOMBRE CLIENTE', 'REFERENCIA ARTÍCULO', 
            'DESCRIPCIÓN ARTÍCULO', 'UNIDAD DE VENTA', 'CANTIDAD SERVIDA', 'PRECIO UNITARIO',
            'IMPORTE', 'IMPORTE IVA', 'TOTAL VENTA','FECHA DE PEDIDO', 'Nº FACTURA', 'FECHA']
        df = clean[cols_ord].copy()


        df['UNIDAD DE VENTA'] = df['CANTIDAD SERVIDA'].copy()
        df['FECHA DE PEDIDO'] = df['FECHA'].copy()
        df['IMPORTE IVA'] = df['IMPORTE'] * 0.21
        df['IMPORTE IVA'] = df['IMPORTE IVA'].round(3)
        df['TOTAL VENTA'] = df['IMPORTE'] + df['IMPORTE IVA']

        #Insertar datos en el excel
        #Creamos copia del archivo plantilla

        global excel_file
        excel_file = 'ReportCompleted.xlsx'

        # Cargar el archivo de plantilla y la hoja específica
        book = load_workbook(plantilla)
        
        #Seleccion plantilla y hoja
        sheet_name = book.sheetnames[0]

        # Seleccionar la hoja donde se va a escribir el DataFrame
        sheet = book[sheet_name]

        # Encontrar la primera columna vacía en la fila 10
        start_col = 1
        while sheet.cell(row=7, column=start_col).value is not None:
            start_col += 1

        # Escribir el DataFrame en el archivo de Excel a partir de la fila 10 y la columna encontrada, sin incluir los nombres de las columnas
        for index, row in df.iterrows():
            for col_num, value in enumerate(row, start=start_col):
                sheet.cell(row=index + 7, column=col_num, value=value)

        # Guardar y cerrar el archivo de Excel
        book.save(excel_file)
        book.close()

        with open("ReportCompleted.xlsx", "rb") as file:

            excel_data = io.BytesIO(file.read())

            btn = st.download_button(
                    label="Descargar",
                    data=excel_data.getvalue(),
                    file_name = 'ReportCompleted.xlsx', 
                    mime = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
                  )
            
    else:
        st.button('Descargar')
        



Overwriting app.py


In [4]:
!streamlit run app.py

^C


In [35]:
import sys
import pkg_resources

# Obtener la versión de Python
print("Versión de Python:", sys.version)

# Obtener la lista de paquetes instalados con sus versiones
installed_packages = pkg_resources.working_set
print("Paquetes instalados:")
for package in installed_packages:
    print(package.key, package.version)

Versión de Python: 3.10.9 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:15) [MSC v.1916 64 bit (AMD64)]
Paquetes instalados:
pywin32 306
pyzmq 25.0.2
packaging 23.0
ipython 8.12.0
jupyter-client 8.1.0
ipykernel 6.22.0
tornado 6.2
psutil 5.9.4
traitlets 5.9.0
jupyter-core 5.3.0
decorator 5.1.1
platformdirs 3.2.0
prompt-toolkit 3.0.38
pygments 2.15.0
python-dateutil 2.8.2
asttokens 2.2.1
six 1.16.0
debugpy 1.6.7
nest-asyncio 1.5.6
executing 1.2.0
jedi 0.18.2
parso 0.8.3
pickleshare 0.7.5
stack-data 0.6.2
colorama 0.4.6
wcwidth 0.2.6
pure-eval 0.2.2
backcall 0.2.0
matplotlib-inline 0.1.6
comm 0.1.3
scikit-learn-intelex 20230228.214818
diff-match-patch 20200713
tzdata 2023.3
daal4py 2023.0.2
certifi 2022.12.7
fsspec 2022.11.0
xarray 2022.11.0
regex 2022.7.9
dask 2022.7.0
distributed 2022.7.0
pytz 2022.7
imagecodecs 2021.8.26
tifffile 2021.7.2
setuptools 65.6.3
cryptography 39.0.1
keyring 23.4.0
conda 23.3.1
boltons 23.0.0
pyopenssl 23.0.0
black 22.6.0
pip 22.3.1
twisted 22.2.0
a